In [38]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from statistics import mean
import re
#For scrapping usage
from urllib.request import urlopen
from bs4 import BeautifulSoup



In [2]:
#using just one hard coded URL for runnung tests
place = "https://www.tripadvisor.com/Restaurant_Review-g303322-d2347734-Reviews-Coco_Bambu_Lago_Sul-Brasilia_Federal_District.html"
page = urlopen(place)
soup = BeautifulSoup(page)

/home/cabral/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/cabral/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
#To check the URL strutcture
#too long, commented it
#print (soup.prettify)

In [4]:
print (soup.title)

<title>Coco Bambu Lago Sul, Brasilia - Restaurant Reviews, Phone Number &amp; Photos - TripAdvisor</title>


In [5]:
print (soup.div)

<div id="fb-root"></div>


In [6]:
#soup.find_all("div")
#Found out too many divs

In [20]:
pricetag = soup.find("div", { "class" : "detail first price_rating separator" })

In [21]:
pricetag = pricetag.string.strip()

In [22]:
pricetag

'$$ - $$$'

In [23]:
#Transforming price tag in intergers 

def translate_symbol_to_numeral(pricetag): 
    pricetag = pricetag.replace(' ', '')
    tags = pricetag.split('-')
    tags = list(map(lambda x : len(x), tags))
    if len(tags) == 1:
        return tags[0]
    else:
        return mean(tags)
pricetag = translate_symbol_to_numeral(pricetag)
print (pricetag)
    

2.5


## Sourcing the meal places

In [26]:
data = pd.read_csv('../data/2016-08-08-last-year.xz',
                   parse_dates=[16],
                   dtype={'document_id': np.str,
                          'congressperson_id': np.str,
                          'congressperson_document': np.str,
                          'term_id': np.str,
                          'cnpj_cpf': np.str,
                          'reimbursement_number': np.str})

In [27]:
meals = data[data.subquota_description == 'Congressperson meal']

In [28]:
grouped = meals.groupby('cnpj_cpf', as_index=False)

print('{} total cnpj/cpfs, {} are unique'.format(len(meals), len(grouped)))

33540 total cnpj/cpfs, 5926 are unique



## Creating a dataframe with the first supplier name for each cnpj_cpf:


In [14]:
cnpj_cpfs = []
names = []
for group in grouped:
    cnpj_cpfs.append(group[0])
    names.append(group[1].iloc[0].supplier)

names = pd.DataFrame({'cnpj_cpf': cnpj_cpfs, 'supplier_name': names})
names.head()

,cnpj_cpf,supplier_name
0,00006453000125,RIPA NA BRASA
1,00009332000137,SORVETERIA E CONFEITARIA DOM BOSCO LTDA
2,00015014000189,Mercado Natural Ltda.
3,00015228000155,Polipan Paes e Doces LTDA
4,00015327000137,YELLOW - COMIDA CASEIRA E PIZZARIA LTDA. - ME


## Isolating the supplier model 

In [15]:
coco_bambu = names.loc[names['supplier_name'] == 'COCO BAMBU LAGO SUL COMERCIO DE ALIMENTOS LTDA.']

In [16]:
coco_bambu

,cnpj_cpf,supplier_name
2668,10542662000147,COCO BAMBU LAGO SUL COMERCIO DE ALIMENTOS LTDA.


## Adding pricetag to the DF

In [17]:
coco_bambu['price_range'] = pricetag

/home/cabral/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
coco_bambu

,cnpj_cpf,supplier_name,price_range
2668,10542662000147,COCO BAMBU LAGO SUL COMERCIO DE ALIMENTOS LTDA.,2.5


## Sourcing Companies dataset
### Will compare CNPJ numbers and retrieve name and fantasy name for future research

In [35]:
companies = pd.read_csv('../data/2016-09-03-companies.xz', dtype={'trade_name': np.str})

/home/cabral/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,22,101,102,103,104,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
companies.iloc[0]

situation_date                                                     03/11/2005
type                                                                   MATRIZ
name                             COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA CAER
phone                                                          (95) 3626-5165
situation                                                               ATIVA
neighborhood                                                        SAO PEDRO
address                                                        R MELVIN JONES
number                                                                    219
zip_code                                                           69.306-610
city                                                                BOA VISTA
state                                                                      RR
opening                                                            21/11/1969
legal_entity                              203-8 - SOCIEDADE DE E

In [49]:
def cleanup_cnpj(cnpj):
    # print(type(cnpj))
    regex = r'\d'
    digits = re.findall(regex, cnpj)
    return ''.join(digits)

cleanup_cnpj('89344324.3241-323')

'893443243241323'

In [50]:
companies['cleaned_cnpj'] = companies['cnpj'].map(cleanup_cnpj)
companies.iloc[0]

situation_date                                                     03/11/2005
type                                                                   MATRIZ
name                             COMPANHIA DE AGUAS E ESGOTOS DE RORAIMA CAER
phone                                                          (95) 3626-5165
situation                                                               ATIVA
neighborhood                                                        SAO PEDRO
address                                                        R MELVIN JONES
number                                                                    219
zip_code                                                           69.306-610
city                                                                BOA VISTA
state                                                                      RR
opening                                                            21/11/1969
legal_entity                              203-8 - SOCIEDADE DE E

In [60]:
congressperson_meal_companies = companies[companies['cleaned_cnpj'].isin(names['cnpj_cpf'])]
# a_dataset[a_dataset['cnpj'].isin(another_data['cnpj])]
congressperson_meal_companies.shape


(5655, 229)

Default Tripadivsor URL for study purpose
https://www.tripadvisor.com/Search?geo=303322&pid=3826&typeaheadRedirect=true&redirect=&startTime=1479311838571&uiOrigin=MASTHEAD&q=coco+bambu&returnTo=__2F__Restaurant__5F__Review__2D__g303322__2D__d2347734__2D__Reviews__2D__Coco__5F__Bambu__5F__Lago__5F__Sul__2D__Brasilia__5F__Federal__5F__District__2E__html&searchSessionId=882DF6A9A9540BDAD8FD36432D65D45B1479311256604ssid

https://www.tripadvisor.com/Search?geo=294280&redirect&q=COCO+BAMBU+FRUTOS+DO+MAR&uiOrigin=MASTHEAD&ssrc=e&typeaheadRedirect=true&returnTo=__2F__Restaurant__5F__Review__2D__g303322__2D__d2347734__2D__Reviews__2D__Coco__5F__Bambu__5F__Lago__5F__Sul__2D__Brasilia__5F__Federal__5F__District__2E__html&pid=3825&startTime=undefined&searchSessionId=882DF6A9A9540BDAD8FD36432D65D45B1479314836242ssid

In [64]:
congressperson_meal_companies[congressperson_meal_companies['cleaned_cnpj'] == '10542662000147'].iloc[0]

situation_date                                                          15/12/2008
type                                                                        MATRIZ
name                                COCO BAMBU LAGO SUL COMERCIO DE ALIMENTOS LTDA
phone                                                               (61) 3561-0025
situation                                                                    ATIVA
neighborhood                                                              BRASILIA
address                          ST CLUBES ESPORTIVOS SUL TRECHO 2 CONJUNTO 36 ...
number                                                                         301
zip_code                                                                70.200-002
city                                                                      BRASILIA
state                                                                           DF
opening                                                                 15/12/2008
lega